# import environments and libraries

In [16]:
#import stuffs
import xlsxwriter, re
from datetime import datetime
from statistics import mean, stdev
import csv

#open file to retreive dates
f = open("TG0615.csv","r")
csv = list(csv.reader(f))

#create workbook and worksheets
workbook = xlsxwriter.Workbook('%s to %s Drum Condensers.xlsx' %(csv[13][2],csv[-1][2]))
bold = workbook.add_format({'bold':True})
worksheet1 = workbook.add_worksheet('TG0615')
worksheet2 = workbook.add_worksheet('TG0608')
worksheet3 = workbook.add_worksheet('TG0616')

# define function to grab timestamps

In [17]:
#function to grab timestamps for each PI
def get_timestamps(name):
    #define lists
    full_list = []
    less_19_5 = []
    less_19_5_cont = []
    time_pairs = []
    greater_than_60 = []
    
    #convert PI values to float
    if name in csv[0]:
        elmt_col = csv[0].index(name)
        full_list.append(name)
        for i, row in enumerate(csv):
            try:
                flt = float(csv[i][elmt_col])
                full_list.append(flt)
            except ValueError:
                pass
     
    #grab indices when PI < 19.5'' H2O
    for i, level in enumerate(full_list):
        try:
            if level < 19.5:
                less_19_5.append(i)
        except TypeError: 
            pass

    #if no values <19.5''H2O, return empty list
    if len(less_19_5) == 0:
        pass
    
    else:
        #remove singular occurances of PI < 19.5'' H2O
        if less_19_5[0] == 1 and full_list[2] < 19.5:
            less_19_5_cont.append(less_19_5[0])
        for indx in less_19_5:
            try:
                if float(full_list[indx-1]) < 19.5 or float(full_list[indx+1]) < 19.5:
                    less_19_5_cont.append(indx)
            except:
                pass
        if less_19_5[-1] == len(full_list)-1 and full_list[-2]< 19.5:
            less_19_5_cont.append(less_19_5[-1])
        
        if len(less_19_5_cont) == 0:
            pass
        else:
            #create list with only start and stop indices for each deviation period
            time_pairs.append(less_19_5_cont[0])
            for i, indx in enumerate(less_19_5_cont):
                try:
                    if (indx - less_19_5_cont[i-1]) > 1 or (less_19_5_cont[i+1] - indx) > 1:
                        time_pairs.append(indx)
                except IndexError:
                    pass
            time_pairs.append(less_19_5_cont[-1])

            #evaluate if each deviation period lasts longer than 1 hr
            for i, element in enumerate(time_pairs):
                if i%2 == 0:
                    try:
                        if (time_pairs[i+1] - element) >= 60:
                            greater_than_60.append(element)
                            greater_than_60.append(time_pairs[i+1])
                    except IndexError:
                        pass

    #return condenser name and time intervals >1hr and PI < 19.5''H2O
    return(name,greater_than_60,less_19_5,full_list,less_19_5_cont,time_pairs)

# write TG0615 tab

In [18]:
import csv
f = open("TG0615.csv","r")
csv = list(csv.reader(f))

#slice csv file
for row in csv:
    del row[0:2]
header = csv[4]
csv = csv[13:]
csv.insert(0,header)

#merge date and time columns into datetime string
for row in csv:
    row[0] = " ".join(row[0:2])
    del row[1]

#regular expressions, list condensers relevant to TG0615
condenser_list = []
for name in csv[0]:
    if re.search("^[PI]", name) is not None and re.search("4209|6606|10027|10028|2036",name) is not None:
        condenser_list.append(name)
#widen columns
worksheet1.set_column('A:A',14)
worksheet1.set_column('B:B',28)
worksheet1.set_column('C:C',2)
worksheet1.set_column('D:E',13)
worksheet1.set_column('F:H',20)
worksheet1.set_column('I:I',18)

#write header
worksheet1.write(0,0, "Condenser", bold)
worksheet1.write(0,1, "TOTAL duration <19.5 (min)", bold)
worksheet1.write(0,3, "Start Index", bold)
worksheet1.write(0,4, "End Index", bold)
worksheet1.write(0,5, "Start Datetime", bold)
worksheet1.write(0,6, "End Datetime", bold)
worksheet1.write(0,7, "Event Duration", bold)
worksheet1.write(0,8, "Average PI Level", bold)
worksheet1.write(0,9, "Std Dev", bold)

result = []
row = 1
for name in condenser_list:
    result = get_timestamps(name)
    full_list = result[3]
    #write condenser name
    worksheet1.write(row,0,result[0])
    #write total duration <19.5
    worksheet1.write(row,1,len(result[2]))
    indices = result[1]
    #write indices 
    if len(indices) != 0:
        for i, element in enumerate(indices):
            if i%2 == 0:
                ele_strt = element
                #start index
                worksheet1.write(row,3,element+13)
                #start datetime
                strt = csv[element][0]
                try:
                    a = datetime.strptime(strt, '%Y-%m-%d %H:%M:%S')
                except:
                    a = datetime.strptime(strt, '%m/%d/%Y %H:%M:%S')
                worksheet1.write(row,5,strt)
            else:
                ele_stp = element
                #stop index
                worksheet1.write(row,4,element+13)
                #stop datetime
                stp = csv[element][0]
                #'%Y-%m-%d %H:%M:%S'
                try:
                    b = datetime.strptime(stp, '%Y-%m-%d %H:%M:%S')
                except:
                    b = datetime.strptime(stp, '%m/%d/%Y %H:%M:%S')
                worksheet1.write(row,6,stp)
                #event duration
                lngth = b - a
                #add 60s to include endpoint
                tot_sec = lngth.total_seconds() + 60
                days = tot_sec // 86400
                hrs = (tot_sec - days*86400) // 3600
                mins = (tot_sec - days*86400 - hrs*3600) // 60
                worksheet1.write(row,7,str(int(days))+"days"+" "+str(int(hrs))+"hrs"+" "+str(int(mins))+"mins")
                #average PI level
                avg_PI = round(mean(full_list[ele_strt:ele_stp+1]),1)
                worksheet1.write(row,8,avg_PI)
                #standard deviation
                std_dev = round(stdev(full_list[ele_strt:ele_stp+1]),1)
                worksheet1.write(row,9,std_dev)
                row += 1
    else:
        worksheet1.write(row,3,"No low level periods over 1hr.")
        row += 1

# write TG0608 tab


In [19]:
import csv
f = open("TG0608.csv","r")
csv = list(csv.reader(f))

#slice csv file
for row in csv:
    del row[0:2]
header = csv[4]
csv = csv[13:]
csv.insert(0,header)

#merge date and time columns into datetime string
for row in csv:
    row[0] = " ".join(row[0:2])
    del row[1]
    
#regular expressions, list condensers relevant to TG0608
condenser_list = []
for name in csv[0]:
    if re.search("^[PI]", name) is not None and re.search("4211|4210|4207|4208",name) is not None:
        condenser_list.append(name)
        
#widen columns
worksheet2.set_column('A:A',14)
worksheet2.set_column('B:B',28)
worksheet2.set_column('C:C',2)
worksheet2.set_column('D:E',13)
worksheet2.set_column('F:H',20)
worksheet2.set_column('I:I',18)

#write header
worksheet2.write(0,0, "Condenser", bold)
worksheet2.write(0,1, "TOTAL duration <19.5 (min)", bold)
worksheet2.write(0,3, "Start Index", bold)
worksheet2.write(0,4, "End Index", bold)
worksheet2.write(0,5, "Start Datetime", bold)
worksheet2.write(0,6, "End Datetime", bold)
worksheet2.write(0,7, "Event Duration", bold)
worksheet2.write(0,8, "Average PI Level", bold)
worksheet2.write(0,9, "Std Dev", bold)

result = []
row = 1
for name in condenser_list:
    result = get_timestamps(name)
    full_list = result[3]
    #write condenser name
    worksheet2.write(row,0,result[0])
    #write total duration <19.5
    worksheet2.write(row,1,len(result[2]))
    indices = result[1]
    #write indices 
    if len(indices) != 0:
        for i, element in enumerate(indices):
            if i%2 == 0:
                ele_strt = element
                #start index
                worksheet2.write(row,3,element+13)
                #start datetime
                strt = csv[element][0]
                try:
                    a = datetime.strptime(strt, '%Y-%m-%d %H:%M:%S')
                except:
                    a = datetime.strptime(strt, '%m/%d/%Y %H:%M:%S')
                worksheet2.write(row,5,strt)
            else:
                ele_stp = element
                #stop index
                worksheet2.write(row,4,element+13)
                #stop datetime
                stp = csv[element][0]
                try:
                    b = datetime.strptime(stp, '%Y-%m-%d %H:%M:%S')
                except:
                    b = datetime.strptime(stp, '%m/%d/%Y %H:%M:%S')
                worksheet2.write(row,6,stp)
                #event duration
                lngth = b - a
                #add 60s to include endpoint
                tot_sec = lngth.total_seconds() + 60
                days = tot_sec // 86400
                hrs = (tot_sec - days*86400) // 3600
                mins = (tot_sec - days*86400 - hrs*3600) // 60
                worksheet2.write(row,7,str(int(days))+"days"+" "+str(int(hrs))+"hrs"+" "+str(int(mins))+"mins")
                #average PI level
                avg_PI = round(mean(full_list[ele_strt:ele_stp+1]),1)
                worksheet2.write(row,8,avg_PI)
                #standard deviation
                std_dev = round(stdev(full_list[ele_strt:ele_stp+1]),1)
                worksheet2.write(row,9,std_dev)
                row += 1
    else:
        worksheet2.write(row,3,"No low level periods over 1hr.")
        row += 1

# write TG0616 tab

In [20]:
import csv
f = open("TG0616.csv","r")
csv = list(csv.reader(f))

#slice csv file
for row in csv:
    del row[0:2]
header = csv[4]
csv = csv[13:]
csv.insert(0,header)

#merge date and time columns into datetime string
for row in csv:
    row[0] = " ".join(row[0:2])
    del row[1]
    
#regular expressions
condenser_list = []
for name in csv[0]:
    if re.search("^[PI]", name) is not None and re.search("1234|2025|2026|2030",name) is not None:
        condenser_list.append(name)
        
#widen columns
worksheet3.set_column('A:A',14)
worksheet3.set_column('B:B',28)
worksheet3.set_column('C:C',2)
worksheet3.set_column('D:E',13)
worksheet3.set_column('F:H',20)
worksheet3.set_column('I:I',18)

#write header
worksheet3.write(0,0, "Condenser", bold)
worksheet3.write(0,1, "TOTAL duration <19.5 (min)", bold)
worksheet3.write(0,3, "Start Index", bold)
worksheet3.write(0,4, "End Index", bold)
worksheet3.write(0,5, "Start Datetime", bold)
worksheet3.write(0,6, "End Datetime", bold)
worksheet3.write(0,7, "Event Duration", bold)
worksheet3.write(0,8, "Average PI Level", bold)
worksheet3.write(0,9, "Std Dev", bold)

result = []
row = 1
for name in condenser_list:
    result = get_timestamps(name)
    full_list = result[3]
    #write condenser name
    worksheet3.write(row,0,result[0])
    #write total duration <19.5
    worksheet3.write(row,1,len(result[2]))
    indices = result[1]
    #write indices 
    if len(indices) != 0:
        for i, element in enumerate(indices):
            if i%2 == 0:
                ele_strt = element
                #start index
                worksheet3.write(row,3,element+13)
                #start datetime
                strt = csv[element][0]
                try:
                    a = datetime.strptime(strt, '%Y-%m-%d %H:%M:%S')
                except:
                    a = datetime.strptime(strt, '%m/%d/%Y %H:%M:%S')
                worksheet3.write(row,5,strt)
            else:
                ele_stp = element
                #stop index
                worksheet3.write(row,4,element+13)
                #stop datetime
                stp = csv[element][0]
                try:
                    b = datetime.strptime(stp, '%Y-%m-%d %H:%M:%S')
                except:
                    b = datetime.strptime(stp, '%m/%d/%Y %H:%M:%S')
                worksheet3.write(row,6,stp)
                #event duration
                lngth = b - a
                #add 60s to include endpoint
                tot_sec = lngth.total_seconds() + 60
                days = tot_sec // 86400
                hrs = (tot_sec - days*86400) // 3600
                mins = (tot_sec - days*86400 - hrs*3600) // 60
                worksheet3.write(row,7,str(int(days))+"days"+" "+str(int(hrs))+"hrs"+" "+str(int(mins))+"mins")
                #average PI level
                avg_PI = round(mean(full_list[ele_strt:ele_stp+1]),1)
                worksheet3.write(row,8,avg_PI)
                #standard deviation
                std_dev = round(stdev(full_list[ele_strt:ele_stp+1]),1)
                worksheet3.write(row,9,std_dev)
                row += 1
    else:
        worksheet3.write(row,3,"No low level periods over 1hr.")
        row += 1
workbook.close()